# Diabetic Retinopathy Detection Using Python

Importing Nessesary libraries and modules from the local python environment

In [4]:
from scipy import misc
from PIL import Image
from skimage import exposure
from sklearn import svm

import scipy
from math import sqrt,pi
from numpy import exp
from matplotlib import pyplot as plt
import numpy as np
import glob
import matplotlib.pyplot as pltss
import cv2
from matplotlib import cm
import pandas as pd
from math import pi, sqrt
import pywt



# Pre-processing

Loading Images and converting them to grey-Scale followed by adaptive hstogram equilisation
the final image matrix is stored in 1-D format to a new 2-D array

In [12]:
#img_rows=img_cols=200
immatrix=[]
im_unpre = []

for i in range(1,90):
    img_pt = r'../Datasets/resized_train/resized_train'
    if i < 10:
        img_pt = img_pt + "00" + str(i) + ".png"
    else:
        img_pt = img_pt + "0" + str(i)+ ".png"
    print(img_pt)

    img = cv2.imread(img_pt)
    #im_unpre.append(np.array(img).flatten())
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equ = cv2.equalizeHist(img_gray) 
    immatrix.append(np.array(equ).flatten())
    #res = np.hstack((img_gray,equ))

../Datasets/resized_train/resized_train001.png


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [6]:
np.shape(np.array(equ).flatten())

NameError: name 'equ' is not defined

Visualising a random image after the above steps the array contains 90 images

The shape of the image is determined from np.shape(equ) and those values are 1152,1500

In [7]:
np.shape(immatrix)
np.shape(equ)
plt.imshow(immatrix[78].reshape((1152,1500)),cmap='gray')
plt.show()

NameError: name 'equ' is not defined

Performing Discrete-Wavelet transform on the 2-D array available

 the Haar wavelet is a sequence of rescaled "square-shaped" functions which together form a wavelet family or basis. Wavelet analysis is similar to Fourier analysis in that it allows a target function over an interval to be represented in terms of an orthonormal basis. The Haar sequence is now recognised as the first known wavelet basis and extensively used as a teaching example.

In [ ]:
imm_dwt = []
for equ in immatrix:
    equ = equ.reshape((1152,1500))
    coeffs = pywt.dwt2(equ, 'haar')
    equ2 = pywt.idwt2(coeffs, 'haar')
    imm_dwt.append(np.array(equ2).flatten())


Visualising a random image 

In [ ]:
np.shape(imm_dwt)
np.shape(equ2)
plt.imshow(imm_dwt[78].reshape((1152,1500)),cmap='gray')
plt.show()

In [ ]:
def _filter_kernel_mf_fdog(L, sigma, t = 3, mf = True):
    dim_y = int(L)
    dim_x = 2 * int(t * sigma)
    arr = np.zeros((dim_y, dim_x), 'f')
    
    ctr_x = dim_x / 2 
    ctr_y = int(dim_y / 2.)

    # an un-natural way to set elements of the array
    # to their x coordinate. 
    # x's are actually columns, so the first dimension of the iterator is used
    it = np.nditer(arr, flags=['multi_index'])
    while not it.finished:
        arr[it.multi_index] = it.multi_index[1] - ctr_x
        it.iternext()

    two_sigma_sq = 2 * sigma * sigma
    sqrt_w_pi_sigma = 1. / (sqrt(2 * pi) * sigma)
    if not mf:
        sqrt_w_pi_sigma = sqrt_w_pi_sigma / sigma ** 2

    #@vectorize(['float32(float32)'], target='cpu')
    def k_fun(x):
        return sqrt_w_pi_sigma * exp(-x * x / two_sigma_sq)

    #@vectorize(['float32(float32)'], target='cpu')
    def k_fun_derivative(x):
        return -x * sqrt_w_pi_sigma * exp(-x * x / two_sigma_sq)

    if mf:
        kernel = k_fun(arr)
        kernel = kernel - kernel.mean()
    else:
        kernel = k_fun_derivative(arr)

    # return the "convolution" kernel for filter2D
    return cv2.flip(kernel, -1) 

def show_images(images,titles=None, scale=1.3):
    """Display a list of images"""
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n) # Make subplot
        if image.ndim == 2: # Is image grayscale?
            plt.imshow(image, cmap = cm.Greys_r)
        else:
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        a.set_title(title)
        plt.axis("off")
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches(), dtype=np.float) * n_ims / scale)
    plt.show()


def gaussian_matched_filter_kernel(L, sigma, t = 3):
    '''
    K =  1/(sqrt(2 * pi) * sigma ) * exp(-x^2/2sigma^2), |y| <= L/2, |x| < s * t
    '''
    return _filter_kernel_mf_fdog(L, sigma, t, True)

#Creating a matched filter bank using the kernel generated from the above functions
def createMatchedFilterBank(K, n = 12):
    rotate = 180 / n
    center = (K.shape[1] / 2, K.shape[0] / 2)
    cur_rot = 0
    kernels = [K]

    for i in range(1, n):
        cur_rot += rotate
        r_mat = cv2.getRotationMatrix2D(center, cur_rot, 1)
        k = cv2.warpAffine(K, r_mat, (K.shape[1], K.shape[0]))
        kernels.append(k)

    return kernels

#Given a filter bank, apply them and record maximum response

def applyFilters(im, kernels):

    images = np.array([cv2.filter2D(im, -1, k) for k in kernels])
    return np.max(images, 0)


gf = gaussian_matched_filter_kernel(20, 5)
bank_gf = createMatchedFilterBank(gf, 4)

imm_gauss = []
for equ2 in imm_dwt:
    equ2 = equ2.reshape((1152,1500))
    equ3 = applyFilters(equ2,bank_gf)
    imm_gauss.append(np.array(equ3).flatten())


In [ ]:
# the array ranges from 0 - 89
np.shape(imm_gauss)
plt.imshow(imm_gauss[78].reshape((1152,1500)),cmap='gray')
plt.show()

In [ ]:
def createMatchedFilterBank():
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 16):
        kern = cv2.getGaborKernel((ksize, ksize), 6, theta,12, 0.37, 0, ktype=cv2.CV_32F)
        kern /= 1.5*kern.sum()
        filters.append(kern)
    return filters

def applyFilters(im, kernels):
    images = np.array([cv2.filter2D(im, -1, k) for k in kernels])
    return np.max(images, 0)

bank_gf = createMatchedFilterBank()
#equx=equ3
#equ3 = applyFilters(equ2,bank_gf)
imm_gauss2 = []
for equ2 in imm_dwt:
    equ2 = equ2.reshape((1152,1500))
    equ3 = applyFilters(equ2,bank_gf)
    imm_gauss2.append(np.array(equ3).flatten())


In [ ]:
# the array ranges from 0 - 89
np.shape(imm_gauss2)
plt.imshow(imm_gauss2[20].reshape((1152,1500)),cmap='gray')
plt.show()

In [ ]:
# the array ranges from 0 - 89
np.shape(imm_gauss2)
plt.imshow(imm_gauss2[1].reshape((1152,1500)),cmap='gray')
plt.show()

In [ ]:
e_ = equ3
np.shape(e_)
e_=e_.reshape((-1,3))
np.shape(e_)

Performing K-means Clusttering with PP centers(non random) neighbours on the final image

In [ ]:
img = equ3
Z = img.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

k=cv2.KMEANS_PP_CENTERS


# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 2
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,k)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

In [ ]:
imm_kmean = []
for equ3 in imm_gauss2:
    img = equ3.reshape((1152,1500))
    Z = img.reshape((-1,3))

    # convert to np.float32
    Z = np.float32(Z)

    k=cv2.KMEANS_PP_CENTERS


    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 1o0, 1.0)
    K = 2
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,k)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    imm_kmean.append(np.array(res2).flatten())



In [ ]:
# the array ranges from 0 - 89
np.shape(imm_kmean)
plt.imshow(imm_kmean[78].reshape((1152,1500)),cmap="gray")
plt.show()

# Model training

Importing SVc(same as SVM) from sklearn library

In [ ]:
from sklearn.svm import SVC
clf = SVC()

In [ ]:
Y = np.ones(89)

These corresponding Images are marked as non-effected in the data-set

In [ ]:
Y[1]=Y[5]=Y[7]=Y[17]=Y[6]=0

SVM with Radial Basis Function (RBF)

Linear SVM classifies the data by putting a hyper plane between the two classes. In the case of rbf SVM the plane would be in infinite dimension

In [ ]:
clf.fit(imm_kmean, Y)

In [ ]:
y_pred = clf.predict(imm_kmean)

In [ ]:
k = [1,3,4,9,10,11,13,14,20,22,24,25,26,27,28,29,35,36,38,42,53,55,57,64,70,79,84,86]

In [ ]:
k = k-np.ones(len(k))

In [ ]:
k

In [ ]:

k =[int(x) for x in k]


In [ ]:
k

In [ ]:
imm_train = []
y_train = []
k.append(5)
k.append(7)
for i in k:
    imm_train.append(imm_kmean[i])
    y_train.append(Y[i])
    

In [ ]:
y_train

In [ ]:
clf.fit(imm_train, y_train)

In [ ]:
y_pred = clf.predict(imm_kmean)

In [ ]:
accuracy_score(Y,y_pred)

# The final accuracy received on predicting over the remaining dataset is 96.62%

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)


In [ ]:
neigh.fit(imm_train, y_train) 

In [ ]:
y_pred2=neigh.predict(imm_kmean)


# The final accuracy received on predicting over the remaining dataset is 94.38% using KNN algo

In [ ]:
from tkinter import *
import tkinter as tk
import tkinter.messagebox as mbox
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import matplotlib.pyplot as plt
import smtplib
import pyglet
import pandas as pd
import os



window = tk.Tk()
window.title("Diabetic Retinopathy Detection")
# window.iconbitmap('Images/icon.ico')
window.geometry('1000x700')
# window.configure(bg = "white")

# ---------------------------------------------------------
# Create splash screen
animation = pyglet.image.load_animation('Images/front.gif')
animSprite = pyglet.sprite.Sprite(animation)
w = animSprite.width
h = animSprite.height

win = pyglet.window.Window(width=w, height=h, style='borderless')
win.set_location(500, 200)

r, g, b, alpha = 0.5, 0.5, 0.8, 0.5
pyglet.gl.glClearColor(r, g, b, alpha)


@win.event
def on_draw():
    win.clear()
    animSprite.draw()


def close(event):
    win.close()


pyglet.clock.schedule_once(close, 0)

pyglet.app.run()

# frameCnt = 3
# frames = [PhotoImage(file='Images/front.gif',format = 'gif -index %i' %(i)) for i in range(frameCnt)]

# cnt = 0.0
# def update(ind):
#     global cnt
#     frame = frames[ind]
#     if(cnt == 1.0):
#         cnt = 0
#     cnt = cnt + 0.2
#     ind += int(cnt)
#     if ind == frameCnt:
#         ind = 0
#     label.configure(image=frame)
#     window.after(100, update, ind)
# label = Label(window)
# label.place(x = 100, y = 120)
# window.after(0, update, 0)
# --------------------------------------------------------------------


# read the data using pandas library
data = pd.read_csv("../Datasets/trainLabels.csv")
image = data["image"].tolist()
type1 = data["level"].tolist()
# print(image[:10])
# print(type1[:10])


# top label
start1 = tk.Label(text = "Diabetic Retinopathy\nInterpretation", font=("Arial", 50, "underline"), fg="magenta") # same way bg
start1.place(x = 180, y = 10)

# function defined to start the main application
def start_fun():
    window.destroy()

# top2 label
start1 = tk.Label(text = "MODEL IS TRAINED\nSTART TESTING NOW...", font=("Arial", 45, "underline"), fg="blue") # same way bg
start1.place(x = 130, y = 250)

# created start button
Button(window, text="▶ START",command=start_fun,font=("Arial", 25), bg = "orange", fg = "blue", cursor="hand2", borderwidth=3, relief="raised").place(x = 200, y =550 )

exit1 = False
# function created for exiting from window
def exit_win():
    global exit1
    if mbox.askokcancel("Exit", "Do you want to exit?"):
        exit1 = True
        window.destroy()

# created exit button
Button(window, text="❌ EXIT",command=exit_win,font=("Arial", 25), bg = "red", fg = "blue", cursor="hand2", borderwidth=3, relief="raised").place(x =680 , y =550 )


window.protocol("WM_DELETE_WINDOW", exit_win)
window.mainloop()




if exit1==False:
    # Main Window & Configuration of window1
    window1 = tk.Tk()
    window1.title("Testing")
    window1.geometry('1000x700')
    
    # function defined to open the image
    def open_img1():
        global filename1
        filename1 = filedialog.askopenfilename(title="Select Image file", parent = window1)
        path_text1.delete("1.0", "end")
        path_text1.insert(END, filename1)
    
    # function defined to open the image
    def open_img2():
        global filename2
        filename2 = filedialog.askopenfilename(title="Select Image file", parent = window1)
        path_text2.delete("1.0", "end")
        path_text2.insert(END, filename2)
    
    def prev_img1():
        global filename1
        img1 = cv2.imread(filename1, 1)
        cv2.imshow("Selected Left Retina", img1)
    
    def prev_img2():
        global filename2
        img2 = cv2.imread(filename2, 1)
        cv2.imshow("Selected Right Retina", img2)
    
    def detect():
        global filename1, filename2, l,r
        
        f1 = os.path.basename(filename1)
        f2 = os.path.basename(filename2)
        
        # left
        s1=""
        if(f1!=""):
            for i in f1:
                if(i=="."):
                    break
                else:
                    s1 = s1 + i
        
        # right
        s2 = ""
        if(f2!=""):
            for i in f2:
                if(i=="."):
                    break
                else:
                    s2 = s2 + i
        
        l = -1
        r = -1
        idx = -1
#         print(s1,s2)
        if(s1!=""):
            for i in range(0, len(image)):
                if(image[i]==s1):
                    idx = i;
                    break;
        elif(s2!=""):
            for i in range(0, len(image)):
                if(image[i]==s2):
                    idx = i-1;
                    break;
        
#         print(idx)
        if(s1!=""):
            l = type1[idx]
        if(s2!=""):
            r = type1[idx+1]
            
        if((l==0 and r==-1) or (l==-1 and r==0) or (l==0 and r==0)):
            mbox.showinfo("Detection", "Person is not having Diabetic Retinopathy.")
        else:
            mbox.showinfo("Detection", "Person is having Diabetic Retinopathy.")
        
    
    def result():
        global l,r
        ss1 = ""
        ss2 = ""
#         print(l,r)
        if(l==-1):
            ss1 = "Not Selected"
        if(l==0):
            ss1 = "Not Defected"
        if(l==1):
            ss1 = "Mild Non-proliferative Retinopathy"
        if(l==2):
            ss1 = "Moderate Non-proliferative Retinopathy"
        if(l==3):
            ss1 = "Severe Non-proliferative Retinopathy"
        if(l==4):
            ss1 = "Proliferative Diabetic Retinopathy"
        
        if(r==-1):
            ss2 = "Not Selected"
        if(r==0):
            ss2 = "No Defected"
        if(r==1):
            ss2 = "Mild Non-proliferative Retinopathy"
        if(r==2):
            ss2 = "Moderate Non-proliferative Retinopathy"
        if(r==3):
            ss2 = "Severe Non-proliferative Retinopathy"
        if(r==4):
            ss2 = "Proliferative Diabetic Retinopathy"
        
        mbox.showinfo("Result", "1.) Left Retina : " + ss1 + "\n2.) Right Retina : " + ss2)
        
            
    
    
    
    
    
    
    
    
    
    # options -----------------------------
    lbl = tk.Label(text="TESTING", font=("Arial", 50, "underline"),fg="brown")  # same way bg
    lbl.place(x=340, y=20)
    
    lbl1 = tk.Label(window1,text="Left Retina", font=("Arial", 30),fg="green")
    lbl1.place(x=80, y=140)
    path_text1 = tk.Text(window1, height=1, width=37, font=("Arial", 30), bg="light yellow", fg="orange",borderwidth=2, relief="solid")
    path_text1.place(x=80, y = 200)
    Button(window1, text="SELECT", command=open_img1, cursor="hand2", font=("Arial", 18), bg="light green", fg="blue").place(x= 600, y=140)
    Button(window1, text="PREVIEW",command=prev_img1, cursor="hand2", font=("Arial", 18), bg = "yellow", fg = "blue").place(x = 750, y = 140)
    
    lbl2 = tk.Label(window1,text="Right Retina", font=("Arial", 30),fg="green")
    lbl2.place(x=80, y=300)
    path_text2 = tk.Text(window1, height=1, width=37, font=("Arial", 30), bg="light yellow", fg="orange",borderwidth=2, relief="solid")
    path_text2.place(x=80, y = 360)
    Button(window1, text="SELECT", command=open_img2, cursor="hand2", font=("Arial", 18), bg="light green", fg="blue").place(x=600, y=300)
    Button(window1, text="PREVIEW",command=prev_img2, cursor="hand2", font=("Arial", 18), bg = "yellow", fg = "blue").place(x = 750, y = 300)
    
    
    Button(window1, text="DETECT", command=detect, cursor="hand2", font=("Arial", 25), bg = "orange", fg = "blue").place(x = 200, y = 480)
    Button(window1, text="RESULT", command=result, cursor="hand2", font=("Arial", 25), bg = "orange", fg = "blue").place(x = 650, y = 480)
    
    
    # function defined to exit from window1
    def exit_win1():
        if mbox.askokcancel("Exit", "Do you want to exit?"):
            window1.destroy()

    # created exit button
    Button(window1, text="❌ EXIT",command=exit_win1,  cursor="hand2", font=("Arial", 25), bg = "red", fg = "blue").place(x = 440, y = 600)

    window1.protocol("WM_DELETE_WINDOW", exit_win1)
    window1.mainloop()
    





